# gpt2 训练案例

In [1]:
# 依赖的包与基础方法封装
import os
import torch
import math
from tqdm import tqdm
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
import torch.nn.utils.rnn as rnn_utils
from datasets import DatasetDict, Dataset

# 使用的设备
device = torch.device('cuda:0')

# 绝对路径获取方法
# curPath = os.path.dirname(os.path.abspath(__file__))
# 在 jupyter 中无法获取到 __file__, 但是可以肯定的是其运行路径是文件所在目录
curPath = os.getcwd()
def getAbsPath (relativePath):
  joinPath = os.path.join(curPath, relativePath)
  return os.path.normpath(
    os.path.abspath(joinPath)
  )

model_path = getAbsPath('../models/gpt2-chitchat-learn')

d:\pyEnvs\ai-quick\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 数据的读取与数据集的制作
这里使用编码器对需要训练的数据进行编码，并格式化为后续训练时会使用到的数据集格式

In [2]:
# 加载编码器
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 读取训练需要的数据
with open(getAbsPath('./train.txt'), 'rb') as f:
  data = f.read().decode('utf-8')

# 需要区分 linux 和 windows 环境下的换行符
if "\r\n" in data:
  all_data = data.split("\r\n\r\n")
else:
  all_data = data.split("\n\n")

val_num = math.floor(len(all_data)/50)  # 1/50 的数据作为验证数据
train_data = all_data[val_num:]
valid_data = all_data[:val_num]

# 特殊字符的编码
sep_id = tokenizer.sep_token_id
cls_id = tokenizer.cls_token_id

# 简化该部分的代码
# 获取对话格式化后的 input_ids
def get_input_ids(dialogue):
  # 将每组对话进一步拆分成一个个句子
  if "\r\n" in data:
    utterances = dialogue.split("\r\n")
  else:
    utterances = dialogue.split("\n")

  input_ids = [cls_id]        # 每个 dialogue 以 [CLS] 开头
  for utterance in utterances:
    input_ids += tokenizer.encode(utterance, add_special_tokens=False)
    input_ids.append(sep_id)  # 每个句子以 [SEP] 分隔

  return input_ids

# 数据集格式定义
train_data_dict = { 'text': [], 'input_ids': [] }
valid_data_dict = { 'text': [], 'input_ids': [] }

# 构建数据集
for index, dialogue in enumerate(tqdm(train_data)):
  input_ids = get_input_ids(dialogue)
  train_data_dict['text'].append(dialogue)
  train_data_dict['input_ids'].append(input_ids)

for index, dialogue in enumerate(tqdm(valid_data)):
  input_ids = get_input_ids(dialogue)
  valid_data_dict['text'].append(dialogue)
  valid_data_dict['input_ids'].append(input_ids)

train_dataset = Dataset.from_dict(train_data_dict)
valid_dataset = Dataset.from_dict(valid_data_dict)

# 创建数据集字典并添加数据集
dataset = DatasetDict()
dataset['train'] = train_dataset
dataset['valid'] = valid_dataset

  0%|          | 0/9801 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:00<00:00, 2992.48it/s]


## 模型加载与配置
这里加载基础模型，并对需要训练的 lora 模型规格做配置，完成配置后再将两者用 peft 做拼接，并用于后续的训练过程

In [3]:
# 加载基础模型
model = AutoModelForCausalLM.from_pretrained(model_path)

# lora 模型部分的设定
peft_config = LoraConfig(
  task_type=TaskType.CAUSAL_LM,
  inference_mode=True,
  r=8,
  lora_alpha=32,
  lora_dropout=0.1,
)

# lora 模型和基础模型的拼接，并在拼接完后移动到 gpu
model = get_peft_model(model, peft_config)
model.to(device)

d:\pyEnvs\ai-quick\Lib\site-packages\peft\tuners\lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(13317, 768)
        (wpe): Embedding(300, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-9): 10 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
          

## 配置对应的训练参数
这里基于 huggingface 自定义的 Trainer 类，自定义对应的训练方法，完成对 gpt2 的 lora 训练

In [ ]:
# 配置自定义的 Trainer
output_dir = getAbsPath('../models/gpt2-chitchat-train')

# 对 Trainer 的继承与自定义
class ModifiedTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    return model.forward(
      input_ids=inputs["input_ids"],
      labels=inputs["labels"],
    ).loss

  def save_model(self, output_dir=None, _internal_call=None):
    # 这里的 output_dir 是按照 epoch 或 step 做过拼接，所以可以直接使用
    self.model.save_pretrained(output_dir)

# dataloader 处理相关
max_len = 150
ignore_index = -100
def data_collator(batch):
  # 只使用数据集中的 input_ids 部分
  batch_input_ids = [i['input_ids'] for i in batch]

  # 长度截取与 tensor 化
  format_input_ids = []
  for idx, input_ids in enumerate(batch_input_ids):
    input_ids = input_ids[:max_len]
    input_ids = torch.tensor(input_ids, dtype=torch.long)
    format_input_ids.append(input_ids)

  # 使用指定的 padding_value 补齐长度不足的部分
  input_ids = rnn_utils.pad_sequence(
    format_input_ids, batch_first=True, padding_value=0
  )
  labels = rnn_utils.pad_sequence(
    format_input_ids, batch_first=True, padding_value=ignore_index
  )
  return {
    "input_ids": input_ids,
    "labels": labels
  }

# 训练参数配置
# TODO 需要补充各个参数的详细含义
output_dir = getAbsPath('../models/gpt2-chitchat-lora')
training_args = TrainingArguments(output_dir)
training_args.set_lr_scheduler(num_epochs=20.0, warmup_ratio=0.1)
training_args.set_optimizer(learning_rate=2.6e-5, epsilon=1.0e-9)
training_args.set_save(strategy='epoch')
# training_args.set_save(steps=20)  # 采用这个在训练开始前进行快速验证
training_args.set_training(num_epochs=20, batch_size=4, gradient_accumulation_steps=4)
training_args.set_logging(report_to=[], strategy='steps', steps=500)

# 需要简化
# 开始训练
trainer = ModifiedTrainer(
  model=model,
  train_dataset=dataset['train'],
  data_collator=data_collator,
  args=training_args
)
trainer.train()